In [1]:
import numpy as np
import json

In [2]:
import utils
import coeurimages_preprocessing as cpro
import lumieres_api as lum
import lumieres_matching as lumatch
import letterboxd_scraping as let
import imdb_scraping as imdb

# LUMIERES API

The id for the lumiere pro api should be stored in a json format in a file named var.env that will be added to .gitignore /!\ do not push a file with the id in the repo using git

The content of the file should look like : 

{

      "username": "mailadress@coe.int",
      "password": "yourpassword"

}


In [3]:
token=lum.get_token()
movie={
    "title": "Microcosmos",
    "director": "claude	nuridsany",
    "production_country": 'FR' ,
    "include_minority_coproducing_country": True,
    "prod_start_year": 1995, #trouve film si prod_year > prod_start_year
}
print((lum.movie_request(movie,token)))
lum.logout(token)

[{'id': 8813, 'imdb_id': 'tt0117040', 'original_title': "Microcosmos : le peuple de l'herbe", 'prod_year': 1996, 'production_countries': 'FR, IT, CH', 'relevance': 0.95, 'matching_title': "Microcosmos : le peuple de l'herbe", 'directors': 'Claude Nuridsany, Marie Pérennou', 'total_admissions_obs': 5877305}]


'{"status":"success"}'

In [4]:
# only providing a dict
lum.find_movie(token,research_params=movie)

[{'id': 8813,
  'imdb_id': 'tt0117040',
  'original_title': "Microcosmos : le peuple de l'herbe",
  'prod_year': 1996,
  'production_countries': 'FR, IT, CH',
  'relevance': 0.95,
  'matching_title': "Microcosmos : le peuple de l'herbe",
  'directors': 'Claude Nuridsany, Marie Pérennou',
  'total_admissions_obs': 5877305}]

In [5]:
# getting the microcosmos admissions
lum.get_admissions(8813)

[{'admissions': 22509,
  'year': 1996,
  'market': 'ES',
  'national': False,
  'region': 'Europe (CoE)'},
 {'admissions': 2001064,
  'year': 1996,
  'market': 'FR',
  'national': True,
  'region': 'Europe (CoE)'},
 {'admissions': 23554,
  'year': 1996,
  'market': 'AT',
  'national': False,
  'region': 'Europe (CoE)'},
 {'admissions': 133004,
  'year': 1996,
  'market': 'BE',
  'national': False,
  'region': 'Europe (CoE)'},
 {'admissions': 351883,
  'year': 1996,
  'market': 'CH',
  'national': False,
  'region': 'Europe (CoE)'},
 {'admissions': 419203,
  'year': 1996,
  'market': 'DE',
  'national': False,
  'region': 'Europe (CoE)'},
 {'admissions': 6500,
  'year': 1996,
  'market': 'LU',
  'national': False,
  'region': 'Europe (CoE)'},
 {'admissions': 278517,
  'year': 1996,
  'market': 'US_CA',
  'national': False,
  'region': 'North America'},
 {'admissions': 864,
  'year': 1996,
  'market': 'RO',
  'national': False,
  'region': 'Europe (CoE)'},
 {'admissions': 955,
  'year': 

In [6]:
# getting microcosmos external ids
lum.get_external_ids(8813)

[{'url': 'https://www.imdb.com/title/tt0117040/',
  'name': 'IMDb ID',
  'id': 'tt0117040'},
 {'url': 'https://web.isan.org/public/en/isan/0000-0001-0A9D-0000-5-0000-0000-M',
  'name': 'ISAN',
  'id': '0000-0001-0A9D-0000-5-0000-0000-M'},
 {'url': 'https://ui.eidr.org/view/content?id=10.5240/3AE3-814F-9A8D-BEA9-5965-A',
  'name': 'EIDR',
  'id': '10.5240/3AE3-814F-9A8D-BEA9-5965-A'},
 {'url': None, 'name': 'Justwatch Id', 'id': 'tm107773'},
 {'url': 'https://www.justwatch.com/it/film/microcosmos-il-popolo-dellerba',
  'name': 'JustWatch title url',
  'id': 'it/film/microcosmos-il-popolo-dellerba'},
 {'url': 'https://www.wikidata.org/wiki/Q675044',
  'name': 'Wikidata',
  'id': 'Q675044'},
 {'url': 'https://www.cbo-boxoffice.com/v4/page000.php3?Xnumitem=110&inc=fichemov.php3&fid=3327',
  'name': 'cbo-boxoffice.com',
  'id': '3327'},
 {'url': 'https://www.cnc.fr/professionnels/visas-et-classification/79747',
  'name': 'CNC (FR)',
  'id': '79747'},
 {'url': None, 'name': 'FFA (DE)', 'id':

# UTILS

In [9]:
matching_result_exemple=[{'recherche': {'title': 'The Hamlet Syndrome (Syndrom Hamleta)',
   'director': 'Elwira Niewiera',
   'production_country': 'PL',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 0.95,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'The Hamlet Syndrome (Syndrom Hamleta)',
   'director': 'Elwira Niewiera',
   'production_country': 'DE',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 0.95,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'The Hamlet Syndrome (Syndrom Hamleta)',
   'director': 'Piotr Rosolowski',
   'production_country': 'PL',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 0.95,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'The Hamlet Syndrome (Syndrom Hamleta)',
   'director': 'Piotr Rosolowski',
   'production_country': 'DE',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 0.95,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'Le Syndrome de Hamlet',
   'director': 'Elwira Niewiera',
   'production_country': 'PL',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 0.95,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'Le Syndrome de Hamlet',
   'director': 'Elwira Niewiera',
   'production_country': 'DE',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 0.95,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'Le Syndrome de Hamlet',
   'director': 'Piotr Rosolowski',
   'production_country': 'PL',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 0.95,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'Le Syndrome de Hamlet',
   'director': 'Piotr Rosolowski',
   'production_country': 'DE',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 0.95,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'The Hamlet Syndrome',
   'director': 'Elwira Niewiera',
   'production_country': 'PL',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 1,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'The Hamlet Syndrome',
   'director': 'Elwira Niewiera',
   'production_country': 'DE',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 1,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'The Hamlet Syndrome',
   'director': 'Piotr Rosolowski',
   'production_country': 'PL',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 1,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]},
 {'recherche': {'title': 'The Hamlet Syndrome',
   'director': 'Piotr Rosolowski',
   'production_country': 'DE',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2020'},
  'resultat': [{'id': 336349,
    'imdb_id': 'tt14618312',
    'original_title': 'The Hamlet Syndrome',
    'prod_year': 2022,
    'production_countries': 'PL, DE',
    'relevance': 1,
    'matching_title': 'The Hamlet Syndrome',
    'directors': 'Elwira Niewiera, Piotr Rosolowski',
    'total_admissions_obs': 2387}]}]

In [10]:
utils.best_id(matching_result_exemple)

{'id': 336349,
 'imdb_id': 'tt14618312',
 'original_title': 'The Hamlet Syndrome',
 'prod_year': 2022,
 'production_countries': 'PL, DE',
 'relevance': 0.95,
 'matching_title': 'The Hamlet Syndrome',
 'directors': 'Elwira Niewiera, Piotr Rosolowski',
 'total_admissions_obs': 2387}

In [11]:
utils.search_params(title=['Neco Jaco Stesti', 'Something Like Happiness', 'Something Like Happiness'],director=['Bohdan Slama'],prod_country=['CZ ', 'DE '],prod_year=2003)

{'title+director+country+year': [{'title': 'Something Like Happiness',
   'director': 'Bohdan Slama',
   'production_country': 'CZ',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2003'},
  {'title': 'Something Like Happiness',
   'director': 'Bohdan Slama',
   'production_country': 'DE',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2003'},
  {'title': 'Neco Jaco Stesti',
   'director': 'Bohdan Slama',
   'production_country': 'CZ',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2003'},
  {'title': 'Neco Jaco Stesti',
   'director': 'Bohdan Slama',
   'production_country': 'DE',
   'include_minority_coproducing_country': True,
   'prod_start_year': '2003'}],
 'title+director+year': [{'title': 'Something Like Happiness',
   'director': 'Bohdan Slama',
   'prod_start_year': '2003'},
  {'title': 'Neco Jaco Stesti',
   'director': 'Bohdan Slama',
   'prod_start_year': '2003'}],
 'director+country+year': [{'directo

# PREPROCESSING

In [15]:
files_data="data/extract_scraping.xlsx"
coproducers_data='data/extract_scraping_coprod.xlsx'
out_file="data/projects_to_be_matched.xlsx"

data=cpro.preprocessing(files_data,coproducers_data,out_file)
data

,ID,Reference,refyear,OriginalTitle,FrenchTitle,EnglishTitle,CommitteDecisionDate,kind,Genre,support,...,budget,reason,Director,AnnouncedAmount,CoproducerId,country1,countryname1_english,percentage_participation,contributor_rank,majmin
0,32,02-050-F,2002,Cert Vi Proc,NaN,The Devil Knows Why,2002-04-24 00:00:00.000,Fiction,NaN,Supported,...,NaN,NaN,[Roman Vavra],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41,02-092-F,2002,True Blue (The Blue Dress),NaN,True Blue,2002-10-02 00:00:00.000,Fiction,NaN,Supported,...,984747.0,NaN,[Yiannis Diamantopoulos],"[196949.0, 787798.0]","[45, 10548]","[BG , GR ]","[Bulgaria, Greece]","[0.2, 0.8]","[2, 1]","[Minority, Majority]"
2,43,02-107-F1,2002,Mathilde,NaN,Mathilde,2002-10-02 00:00:00.000,Fiction,NaN,Supported,...,6800000.0,NaN,[Nina Mimica],"[680000.0, 4080000.0, 680000.0, 1360000.0]","[81, 94, 151, 373]","[DE , IT , ES , UK ]","[Germany, Italy, Spain, United Kingdom]","[0.1, 0.6, 0.1, 0.2]","[3, 1, 3, 2]","[Minority, Majority, Minority, Minority]"
3,45,02-118-F,2002,Waiting for the Clouds,En attendant les nuages,Waiting for the Clouds,2002-11-20 00:00:00.000,Fiction,NaN,Supported,...,NaN,NaN,[Yesim Ustaoglu],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47,02-124-F,2002,Pas sur la bouche,NaN,Not on the Lips,2002-11-20 00:00:00.000,Fiction,NaN,Supported,...,NaN,NaN,[Alain Resnais],NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6023,9721,24-194-C (U),2024,La Ligne Bleue,La Ligne Bleue,The Blue Line,NaN,Documentary,Drama,Inelegible,...,416426.0,NaN,[Marie Dumora],"[329684.0, 86742.0]","[23973, 61029]","[FR , DE ]","[France, Germany]","[0.7917, 0.2083]","[1, 2]","[Majority, Minority]"
6024,9722,24-195-F,2024,Kika,Kika,Kika,NaN,Fiction,Comedy,Inelegible,...,2170258.0,NaN,[Alexe Poukine],"[1565399.0, 604859.0]","[32176, 45689]","[BE , FR ]","[Belgium, France]","[0.7213, 0.2787]","[1, 2]","[Majority, Minority]"
6025,9723,24-196-F,2024,The Proud Princess (Pyšná princezna),La fière princesse,The Proud Princess,NaN,Animation,Drama,Inelegible,...,5022999.0,NaN,"[Radek Beran, David Lisy]","[311302.0, 3959178.0, 752519.0]","[29461, 37564, 61520]","[SK , CZ , PL ]","[Slovakia, Czech Republic, Poland]","[0.06198, 0.78821, 0.14981]","[3, 1, 2]","[Minority, Majority, Minority]"
6026,9724,24-197-F (U),2024,Short Summer,Un Bref Été,Short Summer,NaN,Fiction,Drama,Inelegible,...,1650000.0,NaN,[Nastia Korkia],"[1030300.0, 222100.0, 397600.0]","[37969, 46676, 61039]","[DE , RS , FR ]","[Germany, Serbia, France]","[0.62442, 0.13461, 0.24097]","[1, 3, 2]","[Majority, Minority, Minority]"


In [16]:
data.columns

Index(['ID', 'Reference', 'refyear', 'OriginalTitle', 'FrenchTitle',
       'EnglishTitle', 'CommitteDecisionDate', 'kind', 'Genre', 'support',
       'SecretariatDecision', 'firstfilm', 'secondfilm', 'budget', 'reason',
       'Director', 'AnnouncedAmount', 'CoproducerId', 'country1',
       'countryname1_english', 'percentage_participation', 'contributor_rank',
       'majmin'],
      dtype='object')

In [17]:
# def fun(d):
#     if type(d)==float:
#         print(d)
#         print("/////////////////")
#     else:
#         for didi in d:
#             print(didi)
#         print('-------------')

In [18]:
# def findprob(s):
#     if type(s)==str:
#         if "'" in s or "[" in s or "," in s:
#             print(s)

In [19]:
# def from_str_to_list(s):
#     strip=s[1:-1]
#     spl=[k.strip() for k in strip.split(",")]
#     return spl

In [20]:
# s="[196949.0, 787798.0]"
# from_str_to_list(s)

In [21]:

# d=data.apply(lambda x : findprob(x.Director), axis=1)

In [22]:
# ddd=pd.read_csv("data/projects_to_be_matched.csv")
# ddd["Director"]=ddd.apply(lambda x : from_str_to_list(x.Director), axis=1)
# ddd["test"]=ddd.apply(lambda x : fun(x.Director), axis=1)
# a=ddd.aa

# LUMIERES MATCHING

In [7]:
token=lum.get_token()
lumatch.matching_project("Mario","Thelma","",[],["US"],1950,np.nan,token)

[{'ID': nan,
  'recherche': {'title': 'Mario',
   'production_country': 'US',
   'include_minority_coproducing_country': True,
   'prod_start_year': '1950'},
  'resultat': [{'id': 359282,
    'imdb_id': 'tt6718170',
    'original_title': 'The Super Mario Bros. Movie',
    'prod_year': 2023,
    'production_countries': 'US, JP',
    'relevance': 0.95,
    'matching_title': 'The Super Mario Bros. Movie',
    'directors': 'Aaron Horvath, Michael Jelenic',
    'total_admissions_obs': 35856757},
   {'id': 627,
    'imdb_id': 'tt0068646',
    'original_title': 'The Godfather',
    'prod_year': 1972,
    'production_countries': 'US',
    'relevance': 0.95,
    'matching_title': "Mario Puzo's The Godfather",
    'directors': 'Francis Ford Coppola',
    'total_admissions_obs': 269658},
   {'id': 2160,
    'imdb_id': 'tt0099674',
    'original_title': 'The Godfather: Part III',
    'prod_year': 1990,
    'production_countries': 'US',
    'relevance': 0.95,
    'matching_title': "Mario Puzo's The

Remarque : problème avec les isocodes --> conflit entre UK (Coeurimages) et GB (LUMIERE)
                                                                                                        - Thelma

In [9]:
lumatch.matching_file("data/projects_to_be_matched.xlsx","matching/matching_clean.json",2007,2024,show_progress=True)

  0%|          | 0/18 [00:25<?, ?it/s]


KeyboardInterrupt: 

In [10]:
f = open("matching/matching_clean.json")
data_m = json.load(f) 
f.close() 
lumatch.remove_false_positive(data_m)


FileNotFoundError: [Errno 2] No such file or directory: 'matching/matching_clean.json'

In [ ]:
lumatch.fill_back('data/projects_to_be_matched.xlsx','matching/matching_2017_2019.json','matched/clean_match.xlsx')

c:\Users\CoE TEST PC\Documents\repos\eurimages-dashboard\lumieres_matching.py:88: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filesliced=files[files['refyear']>=start_year ][files['refyear']<=end_year]


,ID,Reference,refyear,OriginalTitle,FrenchTitle,EnglishTitle,CommitteDecisionDate,kind,Genre,support,...,contributor_rank,majmin,lumieres_id,lumieres_title,lumieres_matching_title,lumieres_directors,lumieres_prod_year,lumieres_relevance,lumieres_admissions,imdb_id
0,7963,18-001-F,2018,Nuestras Madres (ex Uspantan),NaN,NaN,2018-03-16 00:00:00.000,Fiction,Drama,Supported,...,"[1, 8]","['Majority', 'Minority']",-1,,,,-1,-1.0,-1,
1,7964,18-002-F,2018,Absolute access,Accès absolu,Absolute access,NaN,Fiction,Action/Fantasy/Genre,Inelegible,...,"[1, 2]","['Majority', 'Minority']",-1,,,,-1,-1.0,-1,
2,7965,18-003-F,2018,Wake,La Veillée,Wake,2018-03-16 00:00:00.000,Fiction,Drama,Rejected,...,"[1, 2]","['Majority', 'Minority']",459330,Tako se je koncalo poletje,Tako se je koncalo poletje,Matjaz Ivanisin,2022,1.0,120,tt27881969
3,7966,18-004-F,2018,Even Mice Belong in Heaven (Myši patrí do nebe),Même les souris vont au paradis,Even Mice Belong in Heaven,2018-03-16 00:00:00.000,Animation,Drama,Supported,...,"[15, 1, 10, 6, 11]","['Minority', 'Majority', 'Minority', 'Minority...",91734,I mysi patrí do nebe,Even Mice Belong in Heaven,"Jan Bubenicek, Denisa Grimmová",2020,1.0,435437,tt3804810
4,7967,18-005-F,2018,Don't Forget To Breathe (Ne pozabi dihati),N’oublie pas de respirer,Don't Forget To Breathe,2018-03-16 00:00:00.000,Fiction,Drama,Rejected,...,"[1, 2]","['Majority', 'Minority']",87815,Ne pozabi dihati,Don't Forget to Breathe,Martin Turk,2019,1.0,8696,tt8644024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,8200,18-233-F,2018,The Translator,Le Traducteur,The Translator,2018-12-13 00:00:00.000,Fiction,Drama,Rejected,...,"[3, 2, 1]","['Minority', 'Minority', 'Majority']",93303,The Translator,Le traducteur,"Rana Kazkaz, Anas Khalaf",2020,1.0,8558,tt10691828
227,8201,18-234-F,2018,Moneyboys,NaN,Moneyboys,2018-12-13 00:00:00.000,Fiction,Drama,Supported,...,"[1, 7, 4]","['Majority', 'Minority', 'Minority']",94174,Moneyboys,Moneyboys,C.B. Yi,2021,1.0,23495,tt11783988
228,8202,18-235-F,2018,Power,Le Pouvoir,Power,2018-12-13 00:00:00.000,Fiction,Drama,Supported,...,"[7, 4, 1]","['Minority', 'Minority', 'Majority']",459704,Moc,Power,Mátyás Prikler,2023,1.0,3114,tt26347282
229,8203,18-236-F,2018,Los Europeos,Les Européens,The Europeans,2018-12-13 00:00:00.000,Fiction,Drama,Rejected,...,"[1, 2]","['Majority', 'Minority']",89454,Los Europeos,Los Europeos,Víctor García León,2020,1.0,1317,tt6681678


# IMDB SCRAPING

In [10]:
imdb.get_awards(imdb.URL_IMDB_CANNES+"2023")#"https://www.imdb.com/event/ev0000982/")

{"Palme d'Or": [{'director/name': 'Justine Triet',
   'title': 'Anatomy of a Fall',
   'original_title': "Anatomie d'une chute",
   'imdb_id': 'tt17009710',
   'winner': True},
  {'director/name': 'Nanni Moretti',
   'title': 'A Brighter Tomorrow',
   'original_title': "Il sol dell'avvenire",
   'imdb_id': 'tt16731908',
   'winner': False},
  {'director/name': 'Nuri Bilge Ceylan',
   'title': 'About Dry Grasses',
   'original_title': 'Kuru Otlar Üstüne',
   'imdb_id': 'tt13231544',
   'winner': False},
  {'director/name': 'Jean-Stéphane Sauvaire',
   'title': 'Asphalt City',
   'original_title': '',
   'imdb_id': 'tt9663896',
   'winner': False},
  {'director/name': 'Wes Anderson',
   'title': 'Asteroid City',
   'original_title': '',
   'imdb_id': 'tt14230388',
   'winner': False},
  {'director/name': 'Ramata-Toulaye Sy',
   'title': 'Banel & Adama',
   'original_title': 'Banel e Adama',
   'imdb_id': 'tt26256406',
   'winner': False},
  {'director/name': 'Jessica Hausner',
   'title'

In [12]:

tit=imdb.get_titles_ratings('data/test_title.tsv')
tit

,averageRating,numVotes
tconst,,
tt0000001,5.7,2102
tt0000002,5.6,282
tt0000003,6.5,2121
tt0000004,5.4,182
tt0000005,6.2,2852
...,...,...
tt9916730,7.0,12
tt9916766,7.1,24
tt9916778,7.2,37


In [13]:
tit.loc["tt17009710",:] # tt17009710 = anatomie d'une chute

averageRating         7.7
numVotes         156954.0
Name: tt17009710, dtype: float64

# LETTERBOXD SCRAPING

In [2]:
let.from_imdb_to_letterboxd('tt16731908')

'https://letterboxd.com/film/spider-man-into-the-spider-verse/'

In [3]:
let.rating('tt16731908')

(4.42, 2641353)

# next